In [1]:
import sys
import os
from dotenv import load_dotenv
load_dotenv()
ROOT_DIR_PATH = os.environ.get('ROOT_PATH')
sys.path.append(os.path.abspath(ROOT_DIR_PATH))  # Adds root directory to sys.path

import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from model.vit import VisionTransformerSmall
from utils.model_io import save_model
from utils.config_loader import load_config
from utils.data_loader import DatasetLoader
from pynvml import (
    nvmlInit, nvmlDeviceGetName, nvmlShutdown,
    nvmlDeviceGetHandleByIndex,
    nvmlDeviceGetMemoryInfo,
    nvmlDeviceGetUtilizationRates
)
from torch.optim.lr_scheduler import CosineAnnealingLR
from timm.data import Mixup
import numpy as np
from transformers import get_cosine_schedule_with_warmup
import wandb
from utils.checkpoints_manager import CheckpointManager
from datetime import datetime as dt
from torch.amp import GradScaler
from torch.amp import autocast
import gc
import subprocess


/home/wd/Documents/work_stuff/ViT_REPLICATION/_vit_rep_py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
DATASET = 'CIFAR100'
DATA_DIR = ''

# loading data
print(f'loading dataset : {DATASET}')
loader = DatasetLoader(dataset_name=DATASET,
                        data_dir=DATA_DIR,
                        batch_size=BATCH,
                        num_workers=NUM_WORKERS,
                        img_size=IMAGE)
train_loader, val_loader = loader.get_loaders()
print(f"Train batches: {len(train_loader)}, Validation batches: {len(val_loader)}")
print('data sanity check')
for images, labels in train_loader:
    print(f'image shape and labels shape in training data - one batch : {images.shape}, {labels.shape}')
    break

In [ ]:

BATCH_SIZE = 128
IMAGE_SIZE = 224
CHANNELS = 3
NUM_STEPS = 100
NUM_WARMUP = 10

model.to(device)
model.eval()

dummy_input = torch.randn(BATCH_SIZE, CHANNELS, IMAGE_SIZE, IMAGE_SIZE).to(device)
#warmup
for _ in range(NUM_WARMUP):
    _ = model(dummy_input)

#benchmark
torch.cuda.synchronize()
start = time.time()
for _ in range(NUM_STEPS):
    _ = model(dummy_input)
    torch.cuda.synchronize()
end = time.time()
total_images = BATCH_SIZE * NUM_STEPS
elapsed_time = end - start
throughput = round(total_images / elapsed_time, 2)
return throughput